# **TP3 - Problema 3**

### **Grupo 23**

Pedro Gonçalves a101250
<br>
José Loureiro a96467
<br>
Bruno Neiva a95311


### **Problema**

Considere de novo o 1º problema do trabalho TP2  relativo à descrição da cifra $$\,\mathsf{A5/1}$$ e o FOTS usando BitVec’s que aì foi definido para a componente do gerador de chaves. Ignore a componente de geração final da chave e restrinja o modelo aos três LFSR’s. 
Sejam $$\,\mathsf{X}_0, \mathsf{X}_1, \mathsf{X}_2\,$$ as variáveis que determinam os estados dos três LFSR’s que ocorrem neste modelo. Como condição inicial  e condição de erro use os predicados
        $$\,\mathsf{I} \ ;\equiv\; (\mathsf{X}_0 > 0)\,\land\,(\mathsf{X}_1 > 0)\,\land\,(\mathsf{X}_2 > 0)\quad$$ e $$\quad \mathsf{E}\;\equiv\;\neg\,\mathsf{I}$$
    
**1.** **Codifique** em “z3” o SFOTS assim definido.


**2.** **Use o algoritmo PDR** “property directed reachability” (codifique-o ou use uma versão pré-existente) e, com ele, tente **provar a segurança** deste modelo.

    


## **Implementação**

Para a resolução deste exercício precisamos de utilizar *BitVectors* para representar vetores de *bits* neste caso, vamos faze uso do tipo de dados **BitVec** já presente na bibilioteca **z3** que pode ser instalada com o comando `pip install z3-solver`, pois incluí métodos base de manipulação sobre estes *BitVectors* como os presentes abaixo. Vamos importar também o **Solver** para verificarmos algumas propriedades do problema.

In [1]:
from pysmt.shortcuts import *


### **Passo 1: Codificação do SFOTS no Z3**

O modelo SFOTS é descrito com:

- *Estado*:   Vetores X0, X1, X2.
- *Condição inicial*:   I ≡ (X0 > 0) ∧ (X1 > 0) ∧ (X2 > 0).
- *Condição de erro*:   E ≡ ¬I.
- *Transições*:   Representam o comportamento dos LFSRs em cada passo.



#### **Código Z3 para o SFOTS:**

In [2]:
def declare(i):

    state = {}

    state['node'] = Symbol('node' + str(i), INT)
    state['x0'] = Symbol('x0' + str(i), BVType(19))
    state['x1'] = Symbol('x1' + str(i), BVType(22))
    state['x2'] = Symbol('x2' + str(i), BVType(23))

    return state

### **Função init**

In [3]:
def init(state):    
    return And(
        Equals(state['x0'], BV(1111, 19)),  
        Equals(state['x1'], BV(11111, 22)),  
        Equals(state['x2'], BV(12222, 23)),
        Equals(state['node'], Int(0))
    )

### **Função err**

In [4]:
def err(state):    # x <= 0
    return Or(
        BVULE(state['x0'], BV(0, 19)),
        BVULE(state['x1'], BV(0, 22)),  
        BVULE(state['x2'], BV(0, 23)),
    )

### **Bits de controlo**

In [5]:
def b_zero (state):

    b0 = BVExtract(state['x0'], 8, 8)
    b1 = BVExtract(state['x1'], 10, 10)
    b2 = BVExtract(state['x2'], 10, 10)

    return Or(Equals(b0,b1), Equals(b0,b2))

def b_one (state):

    b0 = BVExtract(state['x0'], 8, 8)
    b1 = BVExtract(state['x1'], 10, 10)
    b2 = BVExtract(state['x2'], 10, 10)

    return Or(Equals(b1,b0), Equals(b1,b2))

def b_two (state):

    b0 = BVExtract(state['x0'], 8, 8)
    b1 = BVExtract(state['x1'], 10, 10)
    b2 = BVExtract(state['x2'], 10, 10)

    return Or(Equals(b2,b0), Equals(b2,b1))

### **Função transição**

In [6]:
def trans(atual, prox):

    s0 = BV(466944,19)
    s1 = BV(3145728,22)
    s2 = BV(7340160,23)
    

    t0 = And(b_zero(atual), b_one(atual), b_two(atual),
             Equals(prox['x0'], BVXor(BVLShl(atual['x0'], BV(1, 19)), BVXor(s0, atual['x0']))),
             Equals(prox['x1'], BVXor(BVLShl(atual['x1'], BV(1, 22)), BVXor(s1, atual['x1']))),
             Equals(prox['x2'], BVXor(BVLShl(atual['x2'], BV(1, 23)), BVXor(s2, atual['x2'])))
    )

    t1 = And(b_zero(atual), b_one(atual), Not(b_two(atual)),
             Equals(prox['x0'], BVXor(BVLShl(atual['x0'], BV(1, 19)), BVXor(s0, atual['x0']))),
             Equals(prox['x1'], BVXor(BVLShl(atual['x1'], BV(1, 22)), BVXor(s1, atual['x1']))),
             Equals(prox['x2'], atual['x2'])
    )

    t2 = And(b_zero(atual), Not(b_one(atual)), b_two(atual),
            Equals(prox['x0'], BVXor(BVLShl(atual['x0'], BV(1, 19)), BVXor(s0, atual['x0']))),
            Equals(prox['x2'], BVXor(BVLShl(atual['x2'], BV(1, 23)), BVXor(s2, atual['x2']))),
            Equals(prox['x1'], atual['x1'])
    )

    t3 = And(Not(b_zero(atual)), b_one(atual), b_two(atual),
            Equals(prox['x1'], BVXor(BVLShl(atual['x1'], BV(1, 22)), BVXor(s1, atual['x1']))),
            Equals(prox['x2'], BVXor(BVLShl(atual['x2'], BV(1, 23)), BVXor(s2, atual['x2']))),
            Equals(prox['x0'], atual['x0'])
    )

    t4 = And(BVULE(atual['x0'], BV(0, 19)),
            BVULE(atual['x1'], BV(0, 22)),
            BVULE(atual['x2'], BV(0, 23)),
            Equals(prox['x0'], atual['x0']),
            Equals(prox['x1'], atual['x1']),
            Equals(prox['x2'], atual['x2'])
    )
    
    return Or(t0, t1, t2, t3, t4)

In [7]:
def SFOTS(k):
    
    with Solver(name="z3") as s:
        #declara os estados
        trace = [declare(i) for i in range(k+1)]
        
        #adiciona restrição inicial
        s.add_assertion(init(trace[0]))

        #adiciona as transições ao solver
        for i in range(k):
            s.add_assertion(trans(trace[i], trace[i+1]))

        #executa o automato para os k passos
        if s.solve():
            for i in range(k):
                print("Passo", i)
                for v in trace[i]:
                    print(v, "=", s.get_value(trace[i][v]))
                print("-----------")


### **Função pdr**

In [8]:
def pdr(k):
    with Solver(name="z3") as s:
        # Declara os estados
        trace = [declare(i) for i in range(k+1)]
        
        # Condições iniciais
        s.add_assertion(init(trace[0]))
        
        # Condição de erro
        cond_erro = err(trace[k])
        
        # Começa assumindo que a condição de erro não é alcançável
        alcançável = Not(cond_erro)
        s.add_assertion(alcançável)
        
        for i in range(k):
            # Transição do estado i para o estado i+1
            s.add_assertion(trans(trace[i], trace[i+1]))
        
        # Verifica se a condição alcançável se mantém para o último estado
        while True:
            # Verifica se a condição alcançável é satisfatível
            if s.solve():
                # Se satisfatível, significa que podemos alcançar o estado de erro, então precisamos refinar nossa condição alcançável
                # Vamos criar uma nova condição que bloqueia esse caminho
                novo_bloqueio = Or(*[trans(trace[i], trace[i+1]) for i in range(k)])
                alcançável = And(alcançável, Not(novo_bloqueio))
                s.add_assertion(alcançável)
            else:
                # Se insatisfatível, significa que o estado de erro não pode ser alcançado
                print("O estado de erro não é alcançável a partir do estado inicial.")
                break

In [9]:
SFOTS(10)
pdr(10)

Passo 0
node = 0
x0 = 1111_19
x1 = 11111_22
x2 = 12222_23
-----------
Passo 1
node = 0
x0 = 470265_19
x1 = 3177897_22
x2 = 12222_23
-----------
Passo 2
node = 0
x0 = 470265_19
x1 = 2131707_22
x2 = 7368770_23
-----------
Passo 3
node = 0
x0 = 414987_19
x1 = 2131707_22
x2 = 6328390_23
-----------
Passo 4
node = 0
x0 = 384797_19
x1 = 1149709_22
x2 = 6328390_23
-----------
Passo 5
node = 0
x0 = 384797_19
x1 = 171287_22
x2 = 5353546_23
-----------
Passo 6
node = 0
x0 = 82215_19
x1 = 3647289_22
x2 = 5353546_23
-----------
Passo 7
node = 0
x0 = 320361_19
x1 = 2681163_22
x2 = 5353546_23
-----------
Passo 8
node = 0
x0 = 320361_19
x1 = 605149_22
x2 = 184414_23
-----------
Passo 9
node = 0
x0 = 320361_19
x1 = 2837607_22
x2 = 7827554_23
-----------
O estado de erro não é alcançável a partir do estado inicial.
